In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [55]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")

In [71]:
# storing the rate of growth for analysis

#importing previous data
growth_last = pd.read_csv("growth.csv")
# #staging new data to add
growth_df = pd.DataFrame()


growth_df["Date"]=[0]
growth_df["NewCases"]=[0]
growth_df["TotalCases"] =[0]
growth_df["VarianceMultiple"]=[0]
growth_df["GrowthFactor"]=[0]

growth_df["Date"]= dt.datetime.today()
growth_df["NewCases"] = corona_df["NewCases"].sum()
growth_df["TotalCases"] =corona_df["TotalCases"].sum()
growth_df["VarianceMultiple"]= 1 + (growth_df["NewCases"]/growth_df["TotalCases"])
growth_df["GrowthFactor"]=[growth_df["NewCases"][0]/growth_last["NewCases"][len(growth_last)-1]]

#appending new data 
# if growth_df["Date"][0].astype("") > (growth_last["Date"][len(growth_last)-1].astype("date") + 21):
    
#     updated_growth_df = growth_last.append(growth_df)
#     updated_growth_df.reset_index(inplace=True,drop=True)
#     updated_growth_df.to_csv('growth.csv',index = False)
#     updated_growth_df.to_html('growth.html',index = False)
#     updated_growth_df.to_json('growth.json',orient = "records")
updated_growth_df

,Date,NewCases,TotalCases,VarianceMultiple,GrowthFactor
0,2020-03-11 00:57:05.757569,297.0,119220,1.002491,1.000000
1,2020-03-11 01:03:51.872781,297.0,119220,1.002491,1.000000
2,2020-03-11 01:16:11.835765,302.0,119225,1.002533,1.016835
3,2020-03-11 20:18:49.306369,7113.0,126061,1.056425,23.552980
4,2020-03-11 20:38:41.539226,7113.0,126061,1.056425,1.000000


In [72]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)

#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)



df_country_list = df_used["Country"].tolist()
df_used
#df_country_list

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Longitude,Latitude
0,China,80790,12.0,3158.0,0.0,61617.0,16015,4492.0,56.1,103.819074,36.561765
1,Italy,12462,2313.0,827.0,196.0,1045.0,10590,1028.0,206.1,12.070013,42.796626
2,Iran,9000,958.0,354.0,63.0,2959.0,5687,0.0,107.2,54.274070,32.575033
3,S. Korea,7755,242.0,60.0,0.0,288.0,7407,54.0,151.3,127.839161,36.385240
4,France,2281,497.0,48.0,15.0,12.0,2221,105.0,34.9,-2.761729,42.173440
...,...,...,...,...,...,...,...,...,...,...,...
105,Bhutan,1,0.0,0.0,0.0,0.0,1,0.0,0.0,90.401882,27.411066
106,Liechtenstein,1,0.0,0.0,0.0,0.0,1,0.0,0.0,9.535743,47.136658
107,Mongolia,1,0.0,0.0,0.0,0.0,1,0.0,0.3,103.052998,46.826815
108,Togo,1,0.0,0.0,0.0,0.0,1,0.0,0.1,0.962328,8.525314


In [73]:
df_used.to_csv('data.csv')

In [74]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [75]:
corona_json

'[{"Country":"China","TotalCases":80790,"NewCases":12.0,"TotalDeaths":3158.0,"NewDeaths":0.0,"TotalRecovered":61617.0,"ActiveCases":16015,"CriticalCases":4492.0,"Tot\\u00a0Cases\\/1M pop":56.1,"Longitude":103.8190735,"Latitude":36.56176546},{"Country":"Italy","TotalCases":12462,"NewCases":2313.0,"TotalDeaths":827.0,"NewDeaths":196.0,"TotalRecovered":1045.0,"ActiveCases":10590,"CriticalCases":1028.0,"Tot\\u00a0Cases\\/1M pop":206.1,"Longitude":12.07001339,"Latitude":42.79662641},{"Country":"Iran","TotalCases":9000,"NewCases":958.0,"TotalDeaths":354.0,"NewDeaths":63.0,"TotalRecovered":2959.0,"ActiveCases":5687,"CriticalCases":0.0,"Tot\\u00a0Cases\\/1M pop":107.2,"Longitude":54.27407004,"Latitude":32.57503292},{"Country":"S. Korea","TotalCases":7755,"NewCases":242.0,"TotalDeaths":60.0,"NewDeaths":0.0,"TotalRecovered":288.0,"ActiveCases":7407,"CriticalCases":54.0,"Tot\\u00a0Cases\\/1M pop":151.3,"Longitude":127.8391609,"Latitude":36.38523983},{"Country":"France","TotalCases":2281,"NewCases